# Word2Vec實作

*   字詞所代表的意義非常多元，在不同狀況下，會代表不同意思。要把多元意思用單一向量表示，則必須要進行word embedding的動作，也就是把高維向量降為低維向量的過程
*   之前介紹過，利用分散式表示法來表達字詞向量，例如PMI、SVD..統計法..等
*   2013年神經網路盛行後，Tomas Mikolov利用神經網路訓練方式，來獲得字詞的表達向量，獲得很棒的成果。一般認為是利用神經網路模擬人類的理解能力，獲得不錯的分布空間所得到的成果。
*   本範例以維基百科wiki部分資料作範例
*   資料來源：https://dumps.wikimedia.org/zhwiki/20231201/zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2
*   利用結巴分詞(jieba)進行斷詞，gensim套件進行word2vec計算
*   本範例約需1小時長時間執行



In [ ]:
!wget https://dumps.wikimedia.org/zhwiki/20231201/zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2

# opencc是繁簡轉換工具

In [ ]:
!pip install opencc-python-reimplemented

# gensim是訓練word2vec的函式庫

In [ ]:
from gensim.corpora import WikiCorpus

wiki_corpus = WikiCorpus('zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2', dictionary={})

In [ ]:
wiki_corpus

In [ ]:
next(iter(wiki_corpus.get_texts()))[:10]

# 把wiki的資料檔案，轉換成連續文字的txt檔案

In [ ]:
text_num = 0

with open('wiki_text.txt', 'w', encoding='utf-8') as f:
    for text in wiki_corpus.get_texts():
        f.write(' '.join(text)+'\n')
        text_num += 1
        if text_num % 10000 == 0:
            print('{} articles processed.'.format(text_num))

    print('{} articles processed.'.format(text_num))

In [ ]:

import jieba
from opencc import OpenCC


# Initial
cc = OpenCC('s2t')
train_data = open('wiki_text.txt', 'r', encoding='utf-8').read()
train_data = cc.convert(train_data)
train_data = jieba.lcut(train_data)
train_data = [word for word in train_data if word != '']
train_data = ' '.join(train_data)
open('seg.txt', 'w', encoding='utf-8').write(train_data)

In [ ]:
from gensim.models import word2vec


# Settings
seed = 666
sg = 0
window_size = 10
#vector_size = 100
min_count = 1
workers = 8
#epochs = 5
batch_words = 10000

train_data = word2vec.LineSentence('seg.txt')
model = word2vec.Word2Vec(
    train_data,
    min_count=min_count,
    #size=vector_size,
    workers=workers,
    #iter=epochs,
    window=window_size,
    sg=sg,
    seed=seed,
    batch_words=batch_words
)

model.save('word2vec.model')

In [ ]:
from gensim.models import word2vec

string = '電腦'
model = word2vec.Word2Vec.load('word2vec.model')
print(string)

for item in model.wv.most_similar(string):
    print(item)
